# IT media営業利益の回帰分析

## リードジェン営業利益予想

In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# CSVファイルの読み込み
leadgen = pd.read_csv('/workspace/leadgen.csv', index_col=0)

# Dataframe分割
train_lead = leadgen.dropna()
pred_lead = leadgen[leadgen['リード営業利益'].isnull()]

# ダミー変数の作成
train_lead_quarter_dummies = pd.get_dummies(train_lead['四半期'], prefix='quarter')
pred_lead_quarter_dummies = pd.get_dummies(pred_lead['四半期'], prefix='quarter')

# モデルの作成
model_lead = LinearRegression()

# 説明変数と目的変数の設定
X_lead = pd.concat([train_lead[['会員数', 'インターネット広告媒体費', 'IT市況']], train_lead_quarter_dummies], axis=1)
y_lead = train_lead['リード営業利益']

In [4]:
# モデルの学習
model_lead.fit(X_lead, y_lead)

# 予測
pred_lead = pd.concat([pred_lead[['会員数', 'インターネット広告媒体費', 'IT市況']], pred_lead_quarter_dummies], axis=1)
predicted_profit_lead = model_lead.predict(pred_lead)

pred_lead['リード営業利益'] = predicted_profit_lead
pred_lead

,会員数,インターネット広告媒体費,IT市況,quarter_1,quarter_2,quarter_3,quarter_4,リード営業利益
2024-1,117000000,2605593060000,22311950400000,1,0,0,0,2.444912e+08
2024-2,117000000,2605593060000,22311950400000,0,1,0,0,2.684891e+08
2024-3,117000000,2605593060000,22311950400000,0,0,1,0,2.613441e+08
2024-4,117000000,2683760851800,23293676217600,0,0,0,1,2.692413e+08


## メディア営業利益予想

In [5]:
# CSVファイルの読み込み
media = pd.read_csv('/workspace/media.csv', index_col=0)

# Dataframe分割
train_media = media.dropna()
pred_media = media[media['メディア営業利益'].isnull()]

# ダミー変数の作成
train_media_quarter_dummies = pd.get_dummies(train_media['四半期'], prefix='quarter')
pred_media_quarter_dummies = pd.get_dummies(pred_media['四半期'], prefix='quarter')

# モデルの作成
model_media = LinearRegression()

# 説明変数と目的変数の設定
X_media = pd.concat([train_media[['UB', '閲覧数', 'インターネット広告媒体費', 'IT市況']], train_media_quarter_dummies], axis=1)
y_media = train_media['メディア営業利益']

In [6]:
# モデルの学習
model_media.fit(X_media, y_media)

# 予測
pred_media = pd.concat([pred_media[['UB', '閲覧数', 'インターネット広告媒体費', 'IT市況']], pred_media_quarter_dummies], axis=1)
predicted_profit_media = model_media.predict(pred_media)

pred_media['メディア営業利益'] = predicted_profit_media
pred_media

,UB,閲覧数,インターネット広告媒体費,IT市況,quarter_1,quarter_2,quarter_3,quarter_4,メディア営業利益
2024-1,50472727,444545454,2605593060000,22311950400000,1,0,0,0,3.648213e+08
2024-2,50472727,444545454,2605593060000,22311950400000,0,1,0,0,3.688878e+08
2024-3,50472727,444545454,2605593060000,22311950400000,0,0,1,0,4.148294e+08
2024-4,50472727,444545454,2683760851800,23293676217600,0,0,0,1,4.338863e+08


In [7]:
rieki = pred_lead['リード営業利益'] + pred_media['メディア営業利益']
rieki.sum()

2625990453.3112535

# DCF法コード

In [14]:
営業利益 = [3000.00, 3180.0]
法人税率 = 34.5
減価償却 = [300, 300]
投資CF = [100, 100]
金利 = 1.0
リスクプレミアム = 7
ベータ = 1.09
純資産 = 8896257053
株数 = 20875179
wacc = 金利 + ベータ * (リスクプレミアム - 金利)

In [16]:
cf = pd.DataFrame(
    data={
        "営業利益": 営業利益,
        "減価償却": 減価償却,
        "投資CF": 投資CF,
        "FCF": 0,
        "present_val": 0,
        "period": [i for i in range(1, 3)],
    },
    index=["2024", "2025"],
)
cf["営業利益"] = cf["営業利益"] * 1000000
cf["減価償却"] = cf["減価償却"] * 1000000
cf["投資CF"] = cf["投資CF"] * 1000000
cf


,営業利益,減価償却,投資CF,FCF,present_val,period
2024,3.000000e+09,300000000,100000000,0,0,1
2025,3.180000e+09,300000000,100000000,0,0,2


In [17]:
cf["FCF"] = cf["営業利益"] * (1 - 法人税率 / 100) + cf["減価償却"] - cf["投資CF"]
cf.iloc[:-1, cf.columns.get_loc("present_val")] = (
    cf["FCF"].iloc[:-1] / (1 + wacc / 100) ** cf["period"].iloc[:-1]
)
cf.iloc[-1, cf.columns.get_loc("present_val")] = cf["FCF"].iloc[-1] / (wacc - 1) * 100
cf


,営業利益,減価償却,投資CF,FCF,present_val,period
2024,3.000000e+09,300000000,100000000,2.165000e+09,2.013204e+09,1
2025,3.180000e+09,300000000,100000000,2.282900e+09,3.490673e+10,2


In [18]:
想定時価総額 = cf['present_val'].sum() + 純資産
株価 = 想定時価総額 / 株数
株価

2194.768690165996